In [1]:
import json
from pathlib import Path

In [2]:
# Charger le fichier original
input_file = 'prix-carburants-quotidien.json'
output_file = 'prix-carburants-quotidien-cleaned.json'

print(f"Lecture de {input_file}...")
with open(input_file, 'r', encoding='utf-8') as f:
    data = json.load(f)

print(f"Nombre d'entrées chargées: {len(data) if isinstance(data, list) else 'N/A'}")

Lecture de prix-carburants-quotidien.json...
Nombre d'entrées chargées: 74592


In [3]:
# Exemple d'horaire avant transformation
if isinstance(data, list) and len(data) > 0:
    for item in data[:10]:  # Chercher un exemple avec horaires
        if 'horaires' in item and item['horaires']:
            print("Exemple d'horaire AVANT:")
            print(f"Type: {type(item['horaires'])}")
            print(f"Contenu: {item['horaires'][:200]}...")
            break

Exemple d'horaire AVANT:
Type: <class 'str'>
Contenu: {"@automate-24-24": "1", "jour": [{"@id": "1", "@nom": "Lundi", "@ferme": ""}, {"@id": "2", "@nom": "Mardi", "@ferme": ""}, {"@id": "3", "@nom": "Mercredi", "@ferme": ""}, {"@id": "4", "@nom": "Jeudi"...


In [4]:
# Fonction pour transformer les horaires
def parse_horaires(horaires_str):
    """
    Convertit une string JSON échappée en objet JSON.
    Si la conversion échoue, retourne None.
    """
    if not horaires_str or horaires_str == "":
        return None
    
    if isinstance(horaires_str, dict):
        # Déjà un objet JSON
        return horaires_str
    
    try:
        # Parser la string JSON
        return json.loads(horaires_str)
    except (json.JSONDecodeError, TypeError) as e:
        print(f"Erreur de parsing: {e}")
        return None

In [5]:
# Transformer toutes les entrées
processed = 0
errors = 0

if isinstance(data, list):
    for item in data:
        if 'horaires' in item:
            parsed_horaires = parse_horaires(item['horaires'])
            if parsed_horaires is not None:
                item['horaires'] = parsed_horaires
                processed += 1
            elif item['horaires'] != "" and item['horaires'] is not None:
                errors += 1
else:
    # Si c'est un dictionnaire avec des clés
    for key, item in data.items():
        if isinstance(item, dict) and 'horaires' in item:
            parsed_horaires = parse_horaires(item['horaires'])
            if parsed_horaires is not None:
                item['horaires'] = parsed_horaires
                processed += 1
            elif item['horaires'] != "" and item['horaires'] is not None:
                errors += 1

print(f"Horaires transformés: {processed}")
print(f"Erreurs rencontrées: {errors}")

Horaires transformés: 61851
Erreurs rencontrées: 0


In [6]:
# Exemple d'horaire après transformation
if isinstance(data, list) and len(data) > 0:
    for item in data[:10]:
        if 'horaires' in item and item['horaires']:
            print("Exemple d'horaire APRÈS:")
            print(f"Type: {type(item['horaires'])}")
            print(json.dumps(item['horaires'], indent=2, ensure_ascii=False))
            break

Exemple d'horaire APRÈS:
Type: <class 'dict'>
{
  "@automate-24-24": "1",
  "jour": [
    {
      "@id": "1",
      "@nom": "Lundi",
      "@ferme": ""
    },
    {
      "@id": "2",
      "@nom": "Mardi",
      "@ferme": ""
    },
    {
      "@id": "3",
      "@nom": "Mercredi",
      "@ferme": ""
    },
    {
      "@id": "4",
      "@nom": "Jeudi",
      "@ferme": ""
    },
    {
      "@id": "5",
      "@nom": "Vendredi",
      "@ferme": ""
    },
    {
      "@id": "6",
      "@nom": "Samedi",
      "@ferme": ""
    },
    {
      "@id": "7",
      "@nom": "Dimanche",
      "@ferme": ""
    }
  ]
}


In [ ]:
# Sauvegarder le fichier transformé
print(f"Sauvegarde dans {output_file}...")
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

print("✓ Fichier sauvegardé avec succès!")

Sauvegarde dans prix-carburants-quotidien-cleaned.json...


In [ ]:
# Vérification de la taille des fichiers
input_size = Path(input_file).stat().st_size / (1024 * 1024)  # MB
output_size = Path(output_file).stat().st_size / (1024 * 1024)  # MB

print(f"Taille fichier original: {input_size:.2f} MB")
print(f"Taille fichier nettoyé: {output_size:.2f} MB")
print(f"Différence: {output_size - input_size:.2f} MB ({((output_size/input_size - 1) * 100):.1f}%)")